In [1]:
from __future__ import print_function
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import sys
import os
import argparse
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from data import *
from PIL import Image
from utils.augmentations import SSDAugmentation
from layers.functions import Detect
from data import COCO_CLASSES, COCOAnnotationTransform

import torch.utils.data as data
from ssd import build_ssd

/home/broiron/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

save_folder = os.path.join(os.getcwd(), 'test')
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

cuda:0


In [3]:
def test_net(save_folder, net, cuda, testset, transform, thresh):
    # dump predictions and assoc. ground truth to text file for now
    
    detect = Detect(num_classes=2, bkg_label=1, top_k=200, conf_thresh=0.01, nms_thresh=0.445)
    
    filename = os.path.join(save_folder, 'test1.txt')
    num_images = len(testset)
    for i in range(num_images):
        print('Testing image {:d}/{:d}....'.format(i+1, num_images))
        img, annotation, h, w, img_id = testset.pull_item(i)
        print(img_id)
        annotation = testset.pull_anno(i)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        with torch.no_grad():
            x = img.unsqueeze(0)
            x = x.type(torch.FloatTensor)
            x = x.to(device)
            x = Variable(x)

            with open(filename, mode='a') as f:
                f.write('\nGROUND TRUTH FOR: '+str(img_id)+'\n')
                #for box in annotation: 
                #    f.write('label: '+' || '.join(str(b) for b in box)+'\n')
                for box in annotation:
                    f.write('label: '+' || '.join(str(b) for b in box['bbox']) + '|| ' + str(box['category_id'])+'\n')

            net = net.to(device)

            #y = net(x) # forward pass
            y = net.forward(x)
            y = detect.forward(y[0], y[1], y[2])
        detections = y.data    
        # scale each detection back up to the image
        #scale = torch.Tensor([img.shape[1], img.shape[0],
        #                    img.shape[1], img.shape[0]])
        scale = torch.Tensor([w, h, w, h])
        
        pred_num = 0
        for i in range(detections.size(1)):
            j = 0
            while detections[0, i, j, 0] >= 0.6:
                if pred_num == 0:
                    with open(filename, mode='a') as f:
                        f.write('PREDICTIONS: '+'\n')
                score = detections[0, i, j, 0]
                # label_name = CUSTOM_CLASSES[i-1]
                label_name = COCO_CLASSES[i-1]
                pt = (detections[0, i, j, 1:]*scale).cpu().numpy()
                coords = (pt[0], pt[1], pt[2], pt[3])
                pred_num += 1
                with open(filename, mode='a') as f:
                    f.write(str(pred_num)+' label: '+label_name+' score: ' +
                            str(score) + ' '+' || '.join(str(c) for c in coords) + '\n')
                print(str(pred_num)+' label: '+label_name+' score: ' +
                            str(score) + ' '+' || '.join(str(c) for c in coords))
                j += 1


In [4]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
dataset = COCODetection(root='/home/broiron/broiron/line_dataset_vol1_coco/', 
                        transform=SSDAugmentation(300, MEANS), image_set='test',
                       target_transform=COCOAnnotationTransform())

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [6]:
img, annotation, h, w= dataset.pull_item(10)

img.shape


/home/broiron/broiron/model_train/ssd-pytorch-custom/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


ValueError: too many values to unpack (expected 4)

In [7]:
anno = dataset.pull_anno(10)
for box in anno:
    print('label: '+' || '.join(str(b) for b in box['bbox']) + '|| ' + str(box['category_id'])+'\n')


label: 938.25 || 316.61 || 977.87 || 971.6800000000001|| 1

label: 434.62 || 425.95 || 1005.61 || 458.15|| 1

label: 684.76 || 889.08 || 1254.51 || 926.23|| 1



In [8]:
img, anno1, h, w, id = dataset.pull_item(10)

anno2 = dataset.pull_anno(10)
print("--------anno1----------")
print(anno1)
print("--------anno2----------")
print(anno2)

--------anno1----------
[[0.         0.         0.40837796 0.85610996 0.        ]
 [0.02430757 0.         0.79371079 0.43682573 0.        ]
 [0.         0.4419917  0.60232594 1.         0.        ]]
--------anno2----------
[{'area': 25953.8734, 'attributes': {'occluded': False, 'rotation': 0.0}, 'bbox': [938.25, 316.61, 1916.12, 1288.29], 'category_id': 1, 'id': 169, 'image_id': 58, 'iscrowd': 0, 'segmentation': []}, {'area': 18385.877999999993, 'attributes': {'occluded': False, 'rotation': 0.0}, 'bbox': [434.62, 425.95, 1440.23, 884.0999999999999], 'category_id': 1, 'id': 170, 'image_id': 58, 'iscrowd': 0, 'segmentation': []}, {'area': 21166.212499999987, 'attributes': {'occluded': False, 'rotation': 0.0}, 'bbox': [684.76, 889.08, 1939.27, 1815.31], 'category_id': 1, 'id': 171, 'image_id': 58, 'iscrowd': 0, 'segmentation': []}]


In [9]:
scale = np.array([w, h, w, h])

bbox = [938.25, 316.61, 1916.12, 1288.29]
bbox[2] += bbox[0]
bbox[3] += bbox[1]
final = list(np.array(bbox) / scale)
print(final)

[0.4299954170485793, 0.20585825747724318, 1.3081439046746104, 1.0434980494148245]


In [10]:
net = build_ssd(phase='test', size=300, num_classes=2)
net.load_state_dict(torch.load('./weights/linedataset_vol1_1a.pth'))
net.eval()
print("Finished loading model!")

test_net(save_folder, net, True, dataset, BaseTransform(net.size, (104, 117, 123)), 0.6)


Finished loading model!
Testing image 1/100....
5
torch.Size([1, 8732, 2])
1 label: line score: tensor(0.7946) 1325.9536 || 28.157701 || 1808.8206 || 503.22498
Testing image 2/100....
8
torch.Size([1, 8732, 2])
Testing image 3/100....
17
torch.Size([1, 8732, 2])
Testing image 4/100....
21
torch.Size([1, 8732, 2])
Testing image 5/100....
22
torch.Size([1, 8732, 2])
1 label: line score: tensor(0.8157) 1511.2793 || 542.6512 || 2238.4834 || 717.06506
Testing image 6/100....
27
torch.Size([1, 8732, 2])
Testing image 7/100....
34
torch.Size([1, 8732, 2])
Testing image 8/100....
40
torch.Size([1, 8732, 2])
Testing image 9/100....
45
torch.Size([1, 8732, 2])
1 label: line score: tensor(0.7153) 480.8888 || 361.0623 || 516.271 || 516.3693
Testing image 10/100....
54
torch.Size([1, 8732, 2])
Testing image 11/100....
58
torch.Size([1, 8732, 2])
Testing image 12/100....
61
torch.Size([1, 8732, 2])
1 label: line score: tensor(0.7396) 655.9087 || 575.2711 || 1295.0857 || 802.3461
Testing image 13/100

358
torch.Size([1, 8732, 2])
Testing image 72/100....
369
torch.Size([1, 8732, 2])
Testing image 73/100....
373
torch.Size([1, 8732, 2])
Testing image 74/100....
379
torch.Size([1, 8732, 2])
Testing image 75/100....
380
torch.Size([1, 8732, 2])
1 label: line score: tensor(0.7053) 1051.1968 || 1655.5355 || 1185.8236 || 2027.3173
2 label: line score: tensor(0.6707) 625.5484 || 392.54688 || 805.38824 || 750.6601
3 label: line score: tensor(0.6430) 743.68976 || 234.83446 || 960.8168 || 602.82074
Testing image 76/100....
385
torch.Size([1, 8732, 2])
1 label: line score: tensor(0.6191) 269.45172 || 404.26834 || 541.1997 || 598.7088
Testing image 77/100....
388
torch.Size([1, 8732, 2])
1 label: line score: tensor(0.8777) 1032.3704 || -41.434414 || 1359.1345 || 326.73788
2 label: line score: tensor(0.7688) 465.9035 || 1622.0425 || 972.02423 || 1716.5654
3 label: line score: tensor(0.6499) 218.3825 || 1179.7089 || 1339.9445 || 1214.6613
4 label: line score: tensor(0.6114) 279.01608 || 1085.6093